In [1]:
import pandas as pd
df=pd.read_csv("HR_Analytics.csv")
df['BusinessTravel'] = df['BusinessTravel'].str.lower().str.replace('_', '').str.strip()
df['BusinessTravel'] = df['BusinessTravel'].replace({
    'travelrarely': 'Rarely Travel',
    'travel_frequently': 'Travel Frequently',
    'nontravel': 'Non-Travel'
})
df.isnull().sum()
df.dropna(inplace=True) 
df['AgeGroup'] = pd.cut(df['Age'], bins=[18, 30, 40, 60], labels=['<30', '30-40', '40+'])
df['SalarySlab'] = pd.qcut(df['MonthlyIncome'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
print(df['Attrition'].value_counts())


FileNotFoundError: [Errno 2] No such file or directory: 'HR_Analytics.csv'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df['Age'], kde=True)
plt.title('Age Distribution')
plt.show()


In [ ]:
sns.countplot(data=df, x='BusinessTravel', hue='Attrition')
plt.title('Attrition by Business Travel')
plt.xticks(rotation=45)
plt.show()


In [ ]:
sns.countplot(data=df, x='JobSatisfaction')
plt.title('Job Satisfaction Levels')
plt.show()


In [ ]:
sns.countplot(data=df, x='Gender', hue='Attrition')
plt.title('Attrition by Gender')
plt.show()


In [ ]:
sns.boxplot(data=df, x='Attrition', y='MonthlyIncome')
plt.title('Income vs Attrition')
plt.show()


In [ ]:
sns.violinplot(data=df, x='Attrition', y='EnvironmentSatisfaction')
plt.title('Environment Satisfaction and Attrition')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
df.groupby(['BusinessTravel', 'Attrition']).size().unstack().plot(kind='bar', stacked=True)
plt.title('Attrition by Business Travel Type')
plt.ylabel('Number of Employees')
plt.show()


In [5]:
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df_encoded = pd.get_dummies(df, drop_first=True)


NameError: name 'df' is not defined

In [3]:
X = df_encoded.drop('Attrition', axis=1)
y = df_encoded['Attrition']


NameError: name 'df_encoded' is not defined

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'X' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
importances = model.feature_importances_
features = X.columns

# Top 10 most important features
importance_df = pd.DataFrame({'Feature': features, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print(importance_df.head(10))

# Plot
sns.barplot(data=importance_df.head(10), x='Importance', y='Feature')
plt.title("Top Features Influencing Attrition")
plt.show()
